In [10]:
import torch
t = torch.tensor([[[1, 2],
                   [3, 4]],
                  [[5, 6],
                   [7, 8]]])
print(t.size())
print(t.view(t.size(0),-1).shape)
t.view(t.size(0),-1)

2
torch.Size([2, 4])


tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [7]:
from torch.utils.tensorboard import  SummaryWriter
logger = SummaryWriter("log/test")
logger.add_text("Best Acc", str(12), global_step=0)

In [6]:
import numpy as np
import pandas as pd
test_acc = pd.read_csv('exp1_test_acc.csv')
best_error = 100 - test_acc['Value'].max()
best_error

6.830001831054673

In [2]:
from resnet_2 import resnet110
import netron
import torch
from torch.utils import tensorboard
model = resnet110(case=1).cuda()
ts = tensorboard.SummaryWriter(log_dir="./log/model0")

ts.add_graph(model ,torch.randn(128, 3, 36, 36).cuda())
'''torch.save(model.state_dict(), "test.pth")
x = torch.randn(128, 3, 36, 36).cuda()  # 随机生成一个输入
torch.onnx.export(model,x , 'test.pth')  # 将 pytorch 模型以 onnx 格式导出并保存
netron.start('test.pth')  # 输出网络结构

modelData = "./demo.pth"  # 定义模型数据保存的路径
netron.start('test.pth')  # 输出网络结构'''

torch.Size([128, 16, 36, 36])
Layer1: torch.Size([128, 16, 36, 36])
Layer2: torch.Size([128, 32, 18, 18])
Layer3: torch.Size([128, 64, 9, 9])
FC: torch.Size([128, 64])
torch.Size([128, 16, 36, 36])
Layer1: torch.Size([128, 16, 36, 36])
Layer2: torch.Size([128, 32, 18, 18])
Layer3: torch.Size([128, 64, 9, 9])
FC: torch.Size([128, 64])
torch.Size([128, 16, 36, 36])
Layer1: torch.Size([128, 16, 36, 36])
Layer2: torch.Size([128, 32, 18, 18])
Layer3: torch.Size([128, 64, 9, 9])
FC: torch.Size([128, 64])


'torch.save(model.state_dict(), "test.pth")\nx = torch.randn(128, 3, 36, 36).cuda()  # 随机生成一个输入\ntorch.onnx.export(model,x , \'test.pth\')  # 将 pytorch 模型以 onnx 格式导出并保存\nnetron.start(\'test.pth\')  # 输出网络结构\n\nmodelData = "./demo.pth"  # 定义模型数据保存的路径\nnetron.start(\'test.pth\')  # 输出网络结构'

In [2]:
case=1
if case not in {0, 1, 2, 3, 4}:
    raise ValueError('Case only support {0,1,2,3,4}')

ValueError: Case only support {0,1,2,3,4}